## Setup

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras.backend as K
from tqdm import tqdm
import seaborn as sns
import wandb
from wandb.keras import WandbCallback
import keras
from keras.models import Sequential

from pfutils import (get_test_data, get_train_data, get_pseudo_test_data,
                     build_model, get_cosine_annealing_lr_callback, get_fold_indices)

WANDB = True
SUBMIT = False
DATA_GENERATOR = True
TRAIN_ON_BACKWARD_WEEKS = False

#If TEST is False use this to simulate tractable testcases. Should be 0 if SUBMIT = True
PSEUDO_TEST_PATIENTS = 0

In [ ]:
if SUBMIT:
    PSEUDO_TEST_PATIENTS = 0

In [ ]:
# retrieve W&B key
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
wandb_key = user_secrets.get_secret("wandb")
assert wandb_key, "Please create a key.txt or Kaggle Secret with your W&B API key"

#wandb_key = "24020b558f39257d30a084a55cb438922c321495"

!pip install -q --upgrade wandb
!wandb login $wandb_key

## Settings And network

In [ ]:
def build_model(config):
    size = config["NUMBER_FEATURES"]
    actfunc = config["ACTIVATION_FUNCTION"]
    predict_slope = config["PREDICT_SLOPE"]
    drop_out_rate = config["DROP_OUT_RATE"]
    l2_regularization = config["L2_REGULARIZATION"]
    output_normalization = config["OUTPUT_NORMALIZATION"]
    hidden_layers = config["HIDDEN_LAYERS"]
    regularization_constant = config["REGULARIZATION_CONSTANT"]
    drop_out_layers = config["DROP_OUT_LAYERS"]
    Learn_on = config["LEARN_ON"]
    
    if actfunc == 'swish':
        actfunc = tf.keras.activations.swish

    inp = tf.keras.layers.Input(shape=(size), name = "input_features")
    inp2 = tf.keras.layers.Input(shape=(1), name = "slope_FVC")
    inp3 = tf.keras.layers.Input(shape=(1), name = "slope_Weekdiff")
    
    inputs = [inp,inp2,inp3]

    x = inp
    
    for j,n_neurons in enumerate(hidden_layers):
        if l2_regularization:
            x = tf.keras.layers.Dense(n_neurons, activation=actfunc,
                                      kernel_regularizer = tf.keras.regularizers.l2(regularization_constant))(x)
        else:
            x = tf.keras.layers.Dense(n_neurons, activation=actfunc)(x)
        if j in drop_out_layers:
            x = tf.keras.layers.Dropout(drop_out_rate)(x)
    
    FVC_output = tf.keras.layers.Dense(1, name = "FVC_output")(x)
    sigma_output = tf.keras.layers.Dense(1, name = "sigma_output")(x)
    
    if output_normalization:
        FVC_output = tf.math.scalar_mul(tf.constant(50,dtype = 'float32'), FVC_output)
        sigma_output = tf.math.scalar_mul(tf.constant(5,dtype = 'float32'), sigma_output)
        if(not predict_slope):
            FVC_output = tf.math.scalar_mul(tf.constant(100,dtype = 'float32'), FVC_output)
            sigma_output = tf.math.scalar_mul(tf.constant(100,dtype = 'float32'), sigma_output)

    if False:
        FVC_output = tf.add(tf.keras.layers.multiply([FVC_output, inp2]),inp3)
        sigma_output = tf.keras.layers.multiply([sigma_output, inp2])
        
    outputs = tf.keras.layers.concatenate([FVC_output,sigma_output])
    
    def absolute_delta_error(y_true, y_pred):
        y_true = tf.dtypes.cast(y_true, tf.float32)
        y_pred = tf.dtypes.cast(y_pred, tf.float32)
        FVC_true = y_true[:,0]
        
        if(predict_slope):
            slope = y_pred[:,0]
            s = y_pred[:,1]

            weeks_from_start = y_true[:,1]
            FVC_start = y_true[:,2]
            
            sigma = s * weeks_from_start
            # Kan probleem worden by ReLu omdat slope negatief wordt door minimalisering Loss
            FVC_pred = weeks_from_start * slope + FVC_start
        else:
            FVC_pred = tf.abs(y_pred[:,0])
        
        ## ** Hier kan een fout komen doordat de afgeleide moeilijker te berekenen is
        delta = tf.abs(FVC_true - FVC_pred)
        ## **
    
        loss = delta
        return K.mean(loss)


    def sigma_cost(y_true, y_pred):
        y_true = tf.dtypes.cast(y_true, tf.float32)
        y_pred = tf.dtypes.cast(y_pred, tf.float32)
        
        if(predict_slope):
            slope = y_pred[:,0]
            s = y_pred[:,1]

            weeks_from_start = y_true[:,1]
            FVC_start = y_true[:,2]
            
            sigma = s * weeks_from_start
            # Kan probleem worden by ReLu omdat slope negatief wordt door minimalisering Loss
            FVC_pred = weeks_from_start * slope + FVC_start
        else:
            sigma = y_pred[:,1]
        
        sigma_clip = tf.maximum(tf.abs(sigma), 70)
        
        sq2 = tf.sqrt(tf.dtypes.cast(2, dtype=tf.float32))
        loss = tf.math.log(sigma_clip * sq2)
        return K.mean(loss)
    
    def delta_over_sigma(y_true, y_pred):
        y_true = tf.dtypes.cast(y_true, tf.float32)
        y_pred = tf.dtypes.cast(y_pred, tf.float32)
        FVC_true = y_true[:,0]
        
        if(predict_slope):
            slope = y_pred[:,0]
            s = y_pred[:,1]

            weeks_from_start = y_true[:,1]
            FVC_start = y_true[:,2]
            
            sigma = s * weeks_from_start
            # Kan probleem worden by ReLu omdat slope negatief wordt door minimalisering Loss
            FVC_pred = weeks_from_start * slope + FVC_start
        else:
            FVC_pred = tf.abs(y_pred[:,0])
            sigma = tf.abs(y_pred[:,1])
        
        ## ** Hier kan een fout komen doordat de afgeleide moeilijker te berekenen is
        sigma_clip = tf.maximum(tf.abs(sigma), 70)
        delta = tf.abs(FVC_true - FVC_pred)
        delta = tf.minimum(delta, 1000)
        ## **
        
        sq2 = tf.sqrt(tf.dtypes.cast(2, dtype=tf.float32))
        loss = (delta / sigma_clip)*sq2
        return K.mean(loss)
    
    def Laplace_log_likelihood(y_true, y_pred):
        y_true = tf.dtypes.cast(y_true, tf.float32)
        y_pred = tf.dtypes.cast(y_pred, tf.float32)
        FVC_true = y_true[:,0]
        
        if predict_slope:
            slope = y_pred[:,0]
            s = y_pred[:,1]

            weeks_from_start = y_true[:,1]
            FVC_start = y_true[:,2]
            
            sigma = s * weeks_from_start
            # Kan probleem worden by ReLu omdat slope negatief wordt door minimalisering Loss
            FVC_pred = weeks_from_start * slope + FVC_start
        else:
            FVC_pred = tf.abs(y_pred[:,0])
            sigma = tf.abs(y_pred[:,1])
        
        ## ** Hier kan een fout komen doordat de afgeleide moeilijker te berekenen is
        sigma = tf.maximum(tf.abs(sigma), 70)
        delta = tf.abs(FVC_true - FVC_pred)
        ## **
        
        sq2 = tf.sqrt(tf.dtypes.cast(2, dtype=tf.float32))
        loss = (delta / sigma)*sq2 + tf.math.log(sigma * sq2)
        return K.mean(loss)
    
    def Laplace_metric(y_true, y_pred):
        y_true = tf.dtypes.cast(y_true, tf.float32)
        y_pred = tf.dtypes.cast(y_pred, tf.float32)
        FVC_true = y_true[:,0]
        
        if predict_slope:
            slope = y_pred[:,0]
            s = y_pred[:,1]

            weeks_from_start = y_true[:,1]
            FVC_start = y_true[:,2]
            
            sigma = s * weeks_from_start
            # Kan probleem worden by ReLu omdat slope negatief wordt door minimalisering Loss
            FVC_pred = weeks_from_start * slope + FVC_start
        else:
            FVC_pred = tf.abs(y_pred[:,0])
            sigma = tf.abs(y_pred[:,1])
        
        ## ** Hier kan een fout komen doordat de afgeleide moeilijker te berekenen is
        sigma_clip = tf.maximum(tf.abs(sigma), 70)
        delta = tf.abs(FVC_true - FVC_pred)
        delta = tf.minimum(delta, 1000)
        ## **
        
        sq2 = tf.sqrt(tf.dtypes.cast(2, dtype=tf.float32))
        loss = (delta / sigma_clip)*sq2 + tf.math.log(sigma_clip * sq2)
        return K.mean(loss)
    
    if Learn_on:
        opt = tf.keras.optimizers.Adam(learning_rate = 0.0001)
    else: 
        opt = tf.keras.optimizers.Adam()
    model = tf.keras.Model(inputs = inputs, outputs = outputs)    
    
    model.compile(optimizer=opt, loss=Laplace_log_likelihood,
                  metrics = [Laplace_metric, sigma_cost, delta_over_sigma, absolute_delta_error])
    
    return model

In [ ]:
predictions = []

# Number of folds. A number between 1 and 176-PSEUDO_TEST_PATIENTS
FOLDS = 3

#Batch size
BATCH_SIZE = 128

#Dropout rate
DROP_OUT_RATE = 0
DROP_OUT_LAYERS = [] # [0,1,2] voor dropout in de eerste 3 lagen

#Train length
EPOCHS = 250

#L2-Regularization
L2_REGULARIZATION = False
REGULARIZATION_CONSTANT = 0.005

#Input and/or output normalization
INPUT_NORMALIZATION = True
OUTPUT_NORMALIZATION = True
COSINE_CYCLES = 5
MAX_LEARNING_RATE = 5e-4
#Amount of features inputted in NN
NUMBER_FEATURES = 9
USE_GAUSSIAN_ON_FVC = True 
ACTIVATION_FUNCTION = 'swish'

Trial_Layers = [[256], [128], [64], [256,128], [128,64], [64,64], [128,64,64]]
Trial_Gauss = [0,70,120,180]
Trial_Correlating_Gauss = [0,1]
Trial_Forwards = [0,1]
Learning_rate_automatic = [0,1]

STEPS_PER_EPOCH = 100
PREDICT_SLOPE = 0

for qqq in range(200):
    HIDDEN_LAYERS = Trial_Layers[np.random.randint(7)]
    VALUE_GAUSSIAN_NOISE_ON_FVC = Trial_Gauss[np.random.randint(4)]
    GAUSSIAN_NOISE_CORRELATED = np.random.rand(1) > 0.5
    LEARN_ON = np.random.rand(1) > 0.5
    FORWARD_ONLY = np.random.rand(1) > 0.5

    MODEL_NAME = "Random: " + str(qqq)

    config = dict(NUMBER_FEATURES = NUMBER_FEATURES, L2_REGULARIZATION = L2_REGULARIZATION, INPUT_NORMALIZATION = INPUT_NORMALIZATION,
              ACTIVATION_FUNCTION = ACTIVATION_FUNCTION, DROP_OUT_RATE = DROP_OUT_RATE, OUTPUT_NORMALIZATION = OUTPUT_NORMALIZATION,
              EPOCHS = EPOCHS, STEPS_PER_EPOCH = STEPS_PER_EPOCH, MAX_LEARNING_RATE = MAX_LEARNING_RATE,
              COSINE_CYCLES = COSINE_CYCLES, MODEL_NAME=MODEL_NAME, USE_GAUSSIAN_ON_FVC=USE_GAUSSIAN_ON_FVC,
              VALUE_GAUSSIAN_NOISE_ON_FVC=VALUE_GAUSSIAN_NOISE_ON_FVC, PREDICT_SLOPE = PREDICT_SLOPE,
              HIDDEN_LAYERS = HIDDEN_LAYERS, REGULARIZATION_CONSTANT = REGULARIZATION_CONSTANT,
              DROP_OUT_LAYERS = DROP_OUT_LAYERS, BATCH_SIZE = BATCH_SIZE, GAUSSIAN_NOISE_CORRELATED = GAUSSIAN_NOISE_CORRELATED, 
              LEARN_ON = LEARN_ON, FORWARD_ONLY = FORWARD_ONLY)
    
    train, data, labels = get_train_data('../input/osic-pulmonary-fibrosis-progression/train.csv', PSEUDO_TEST_PATIENTS, INPUT_NORMALIZATION, TRAIN_ON_BACKWARD_WEEKS)
    
    fold_pos = get_fold_indices(FOLDS, train)
    
    class DataGenerator(keras.utils.Sequence):
        def __init__(self, list_IDs, config, validation = False, number_of_labels = 3,
                     batch_size = 128, shuffle = True):
            self.number_features = int(config["NUMBER_FEATURES"])
            self.validation = validation
            self.use_gaussian = config["USE_GAUSSIAN_ON_FVC"]
            self.gauss_std = config["VALUE_GAUSSIAN_NOISE_ON_FVC"] and not validation
            self.list_IDs = list_IDs
            self.batch_size = config["BATCH_SIZE"]
            self.labels = labels
            self.shuffle = shuffle
            self.on_epoch_end()
            self.label_size = number_of_labels
            self.normalized = config["INPUT_NORMALIZATION"]
            self.correlated = config["GAUSSIAN_NOISE_CORRELATED"]

        def __len__(self):
            return int(np.floor(len(self.list_IDs)/self.batch_size))

        def __getitem__(self, index):
            'Generate one batch of data'
            # Generate indexes of the batch
            indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
            list_IDs_temp = [self.list_IDs[k] for k in indexes]
            # Generate data
            X, y = self.__data_generation(list_IDs_temp)
            return X, y

        def on_epoch_end(self):
            'Updates indexes after each epoch'
            self.indexes = np.arange(len(self.list_IDs))
            if self.shuffle == True:
                np.random.shuffle(self.indexes)

        def __data_generation(self, list_IDs_temp):
            'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
            # Initialization
            X = np.empty((self.batch_size, self.number_features))
            y = np.empty((self.batch_size, self.label_size), dtype=int)

            data = np.load("./train_data.npy", allow_pickle = True)
            lab = np.load("./train_labels.npy", allow_pickle = True)

            for i, ID in enumerate(list_IDs_temp):
                X[i,] = np.asarray(data[ID], dtype = "float32")
                y[i,] = np.asarray(lab[ID], dtype = "float32")
            y = np.asarray(y,dtype = "float32")

            gauss_X = np.random.normal(0, self.gauss_std, size = self.batch_size)

            if self.correlated:
                gauss_y = gauss_X
            else:
                gauss_y = np.random.normal(0, self.gauss_std, size = self.batch_size)
            if self.normalized:
                gauss_X = gauss_X/5000 

            if self.validation: 
                gauss_X = np.random.normal(0, 0, size = self.batch_size)
                gauss_y = np.random.normal(0, 0, size = self.batch_size)
            
            X[:,2] += gauss_X.astype("float32")*X[:,2]/X[:,1]
            X[:,1] += gauss_X.astype("float32")
            y[:,2] += gauss_X.astype("float32")
            y[:,0] += gauss_y.astype("float32")

            return X, y
    
    if DATA_GENERATOR:
        train_data = train[["Weeks", "FVC", "Percent", "Age", "Sex", 
                                     "Currently smokes", "Ex-smoker", "Never smoked", "Weekdiff_target"]]
        train_labels = labels
        np.save("train_data.npy", train_data.to_numpy())
        np.save("train_labels.npy", train_labels.to_numpy())
    
    for fold in range(FOLDS):
        if DATA_GENERATOR:
            train_ID = list(range(fold_pos[0],fold_pos[fold])) + list(range(fold_pos[fold+1],len(train)))
            val_ID = list(range(fold_pos[fold], fold_pos[fold+1]))
            # Generators
            training_generator = DataGenerator(train_ID, config)
            validation_generator = DataGenerator(val_ID, config, validation = True)
        else:
            x_train = data["input_features"][:fold_pos[fold]].append(data["input_features"][fold_pos[fold+1]:])
            y_train = labels[:fold_pos[fold]].append(labels[fold_pos[fold+1]:])
            x_val = data["input_features"][fold_pos[fold]:fold_pos[fold+1]]
            y_val = labels[fold_pos[fold]:fold_pos[fold+1]]

        model = build_model(config)

        sv = tf.keras.callbacks.ModelCheckpoint(
        'fold-%i.h5'%fold, monitor='val_loss', verbose=0, save_best_only=True,
        save_weights_only=True, mode='min', save_freq='epoch')
        callbacks = [sv]

        print(fold+1, "of", FOLDS)
        if WANDB:
            name = MODEL_NAME + '-F{}'.format(fold+1)
            config.update({'fold': fold+1})
            wandb.init(project="pulfibrandom2", name=name, config=config)
            wandb_cb = WandbCallback()
            callbacks.append(wandb_cb)

        if DATA_GENERATOR:
            history = model.fit(training_generator, validation_data = validation_generator, epochs = EPOCHS,
                                verbose = 0, callbacks = callbacks)
        else:
            history = model.fit(x_train, y_train, validation_data = (x_val,y_val), epochs = EPOCHS,
                                steps_per_epoch = STEPS_PER_EPOCH, verbose = 0, callbacks = callbacks)

        if SUBMIT or PSEUDO_TEST_PATIENTS > 0:
            model.load_weights('fold-%i.h5'%fold)
            predictions.append(model.predict(test_data, batch_size = 256))

        if WANDB:
            # finalize run
            wandb.join()